In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
from bs4 import BeautifulSoup
import mysql.connector
import pandas as pd

In [2]:
# Common function defenition
def connect_db():
    db_connection = mysql.connector.connect(
    host='146.56.43.43',
    user='ffbemaster',
    password='aksaksgo1!',
    database='ffbe')
    return db_connection
def fetch_data_from_db(sql):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute(sql)
    res = cursor.fetchall()
    cursor.close()
    conn.close()
    return res
def fetch_data_from_db_with_col_names(sql):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute(sql)
    res = cursor.fetchall()
    col_names = cursor.column_names
    cursor.close()
    conn.close()
    return res, col_names
def write_to_server(sql, values=None):
    conn = connect_db()
    cursor = conn.cursor()
    if values == None:
        cursor.execute(sql)
    else:
        if type(values) != list:
            cursor.execute(sql, values)
        else:
            for i in range(len(values)):
                cursor.execute(sql, values[i])
    res = cursor.fetchall()
    cursor.close()
    conn.commit()
    conn.close()
    return res

In [17]:
# scrapig function definition
class VC_SCRAPER():
    def __init__(self):
        # variable setting
        self.table_name = 'vc_for_job_for_update_list'
        self.excel_file_name = "vc_for_job_from_web.xlsx"
        self.values_list = []
    def load_chrome_driver(self):
        # Automatically download and install the latest version of ChromeDriver
        chromedriver_autoinstaller.install()
        # VC_for_job_update
        self.driver = webdriver.Chrome()
        self.driver.get('https://game8.jp/geneisensou/299436')
        job_vc_btn = self.driver.find_element(By.CSS_SELECTOR, 'body > div.l-content.js-end-trigger > div.l-3col > div.l-3colMain > div.l-3colMain__center.l-3colMain__center--shadow > div.archive-style-wrapper > div.a-tabContainer > div.a-tabs > div:nth-child(2)')
        job_vc_btn.click()
    def scrape_vc_for_job_as_jpn(self):
        soup = BeautifulSoup(self.driver.page_source, 'html.parser')

        tbody = soup.select('body > div.l-content.js-end-trigger > div.l-3col > div.l-3colMain > div.l-3colMain__center.l-3colMain__center--shadow > div.archive-style-wrapper > div.a-tabContainer > div.a-tabPanels > div.a-tabPanel.is-active > table > tbody')
        vctrs = tbody[0].select('tbody > tr')

        for cnt in range(0,int(len(vctrs)/7)):
            start = 7*cnt
            end = 7*(cnt+1)
            vctr = vctrs[start:end]
            name_tr = vctr[1]
            
            vc_jp_name = name_tr.find('td').getText()
            vc_g8_link = name_tr.find(class_='a-link')['href']
            vc_img_src = name_tr.find('img')['src']
            values = [vc_jp_name, vc_g8_link, vc_img_src]
            
            class_tds = vctr[2].findAll('td')
            if class_tds[0].getText() == '◯':
                # print("검1")
                values += ("1",)
            else:
                values += ("0",)
            if class_tds[1].getText() == '◯':
                # print("검2")
                values += ("1",)
            else:
                values += ("0",)
            if class_tds[2].getText() == '◯':
                # print("검3")                
                values += ("1",)
            else:
                values += ("0",)    
            if class_tds[3].getText() == '◯':
                # print("지팡이1")                
                values += ("1",)
            else:
                values += ("0",)
            if class_tds[4].getText() == '◯':
                # print("지팡이2")                
                values += ("1",)
            else:
                values += ("0",)
            if class_tds[5].getText() == '◯':
                # print("대검")                
                values += ("1",)
            else:
                values += ("0",)
            
            class_tds = vctr[4].findAll('td')
            if class_tds[0].getText() == '◯':
                # print("창")
                values += ("1",)
            else:
                values += ("0",)
            if class_tds[1].getText() == '◯':
                # print("도끼")
                values += ("1",)
            else:
                values += ("0",)
            if class_tds[2].getText() == '◯':
                # print("활")                
                values += ("1",)
            else:
                values += ("0",)
            if class_tds[3].getText() == '◯':
                # print("총")                
                values += ("1",)
            else:
                values += ("0",)
            if class_tds[4].getText() == '◯':
                # print("주먹")                
                values += ("1",)
            else:
                values += ("0",)
            if class_tds[5].getText() == '◯':
                # print("단검")                
                values += ("1",)
            else:
                values += ("0",)    

            class_tds = vctr[6].findAll('td')
            if class_tds[0].getText() == '◯':
                # print("닌자")
                values += ("1",)
            else:
                values += ("0",)
            if class_tds[1].getText() == '◯':
                # print("도검")
                values += ("1",)
            else:
                values += ("0",)
            if class_tds[2].getText() == '◯':
                # print("메이스")                
                values += ("1",)
            else:
                values += ("0",)    
            if class_tds[3].getText() == '◯':
                # print("글로브")                
                values += ("1",)
            else:
                values += ("0",)
            if class_tds[4].getText() == '◯':
                # print("책")                
                values += ("1",)
            else:
                values += ("0",)
            if class_tds[5].getText() == '◯':
                # print("부메랑")
                values += ("1",)
            else:
                values += ("0",)        
            # print(values)
            # End of scraping a CV
            # Write to server
            self.values_list.append(values)
        print(len(self.values_list))
        print(self.values_list[0])
        print(self.values_list[-1])
        
    def scrape_kor_name(self):
        # 한글 번역 이름 scraping 수동으로 번역하기
        # 수동으로 한글로 번역하여 홈페이지를 로드하자. 
        soup = BeautifulSoup(self.driver.page_source, 'html.parser')

        tbody = soup.select('body > div.l-content.js-end-trigger > div.l-3col > div.l-3colMain > div.l-3colMain__center.l-3colMain__center--shadow > div.archive-style-wrapper > div.a-tabContainer > div.a-tabPanels > div.a-tabPanel.is-active > table > tbody')
        vctrs = tbody[0].select('tbody > tr')
        vc_trsl_name_list = []
        for cnt in range(0, int(len(vctrs)/7)):
            start = 7*cnt
            end = 7*(cnt+1)
            vctr = vctrs[start:end]
            name_tr = vctr[1]
            vc_trsl_name_list.append(name_tr.find('td').getText())
        for i in range(len(vc_trsl_name_list)):
            self.values_list[i] = (vc_trsl_name_list[i], *self.values_list[i])
        print(len(vc_trsl_name_list))
        print(self.values_list[0])
        print(self.values_list[-1])
    
    def upload_to_server(self):
        # update VC for JOB to server.
        sql = f"""INSERT INTO `ffbe`.`{self.table_name}`
            (`vc_trsl_name`,`vc_jp_name`,`vc_g8_link`, `vc_img_src`,`검1`,`검2`,`검3`,`지팡이1`,`지팡이2`,`대검`,`창`,`도끼`,`활`,`총`,`주먹`,`단검`,`닌자`,`도검`,`메이스`,`글로브`,`책`,`부메랑`) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
        write_to_server(sql,self.values_list)
        # 서버에 올라간 내용을 엑셀파일로도 저장.
        sql_query = f"SELECT * from {self.table_name}"
        data = fetch_data_from_db(sql_query)
        df = pd.DataFrame(data)
        df.to_excel(self.excel_file_name, index=False)


In [20]:
vc_scraper = VC_SCRAPER()
vc_scraper.load_chrome_driver()

In [21]:
vc_scraper.scrape_vc_for_job_as_jpn()

52
['浜辺の勝者', 'https://game8.jp/geneisensou/543354', 'https://img.game8.jp/8459955/230af791f47da2d6888d6cfed2e122b7.png/show', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '1', '0']
['リオニス旗', 'https://game8.jp/geneisensou/301656', 'data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7', '0', '1', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0']


In [22]:
vc_scraper.scrape_kor_name()
vc_scraper.upload_to_server()
#VC saraper 끝!!

52
('해변의 승자', '浜辺の勝者', 'https://game8.jp/geneisensou/543354', 'https://img.game8.jp/8459955/230af791f47da2d6888d6cfed2e122b7.png/show', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '1', '0')
('리오니스 국기', 'リオニス旗', 'https://game8.jp/geneisensou/301656', 'data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7', '0', '1', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0')


In [3]:
# VC_for_job_update
driver = webdriver.Chrome()
driver.get('https://game8.jp/geneisensou/299436')
job_vc_btn = driver.find_element(By.CSS_SELECTOR, 'body > div.l-content.js-end-trigger > div.l-3col > div.l-3colMain > div.l-3colMain__center.l-3colMain__center--shadow > div.archive-style-wrapper > div.a-tabContainer > div.a-tabs > div:nth-child(2)')
job_vc_btn.click()

Error getting version of chromedriver 115. Retrying with chromedriver 114 (attempt 1/5)


In [ ]:
# variable setting
table_name = 'vc_for_job_for_update_list'

soup = BeautifulSoup(driver.page_source, 'html.parser')

tbody = soup.select('body > div.l-content.js-end-trigger > div.l-3col > div.l-3colMain > div.l-3colMain__center.l-3colMain__center--shadow > div.archive-style-wrapper > div.a-tabContainer > div.a-tabPanels > div.a-tabPanel.is-active > table > tbody')
vctrs = tbody[0].select('tbody > tr')

values_list = []
for cnt in range(0,int(len(vctrs)/7)):
    start = 7*cnt
    end = 7*(cnt+1)
    vctr = vctrs[start:end]
    name_tr = vctr[1]
    
    vc_jp_name = name_tr.find('td').getText()
    vc_g8_link = name_tr.find(class_='a-link')['href']
    vc_img_src = name_tr.find('img')['src']
    values = [vc_jp_name, vc_g8_link, vc_img_src]
    
    class_tds = vctr[2].findAll('td')
    if class_tds[0].getText() == '◯':
        # print("검1")
        values += ("1",)
    else:
        values += ("0",)
    if class_tds[1].getText() == '◯':
        # print("검2")
        values += ("1",)
    else:
        values += ("0",)
    if class_tds[2].getText() == '◯':
        # print("검3")                
        values += ("1",)
    else:
        values += ("0",)    
    if class_tds[3].getText() == '◯':
        # print("지팡이1")                
        values += ("1",)
    else:
        values += ("0",)
    if class_tds[4].getText() == '◯':
        # print("지팡이2")                
        values += ("1",)
    else:
        values += ("0",)
    if class_tds[5].getText() == '◯':
        # print("대검")                
        values += ("1",)
    else:
        values += ("0",)
    
    class_tds = vctr[4].findAll('td')
    if class_tds[0].getText() == '◯':
        # print("창")
        values += ("1",)
    else:
        values += ("0",)
    if class_tds[1].getText() == '◯':
        # print("도끼")
        values += ("1",)
    else:
        values += ("0",)
    if class_tds[2].getText() == '◯':
        # print("활")                
        values += ("1",)
    else:
        values += ("0",)
    if class_tds[3].getText() == '◯':
        # print("총")                
        values += ("1",)
    else:
        values += ("0",)
    if class_tds[4].getText() == '◯':
        # print("주먹")                
        values += ("1",)
    else:
        values += ("0",)
    if class_tds[5].getText() == '◯':
        # print("단검")                
        values += ("1",)
    else:
        values += ("0",)    

    class_tds = vctr[6].findAll('td')
    if class_tds[0].getText() == '◯':
        # print("닌자")
        values += ("1",)
    else:
        values += ("0",)
    if class_tds[1].getText() == '◯':
        # print("도검")
        values += ("1",)
    else:
        values += ("0",)
    if class_tds[2].getText() == '◯':
        # print("메이스")                
        values += ("1",)
    else:
        values += ("0",)    
    if class_tds[3].getText() == '◯':
        # print("글로브")                
        values += ("1",)
    else:
        values += ("0",)
    if class_tds[4].getText() == '◯':
        # print("책")                
        values += ("1",)
    else:
        values += ("0",)
    if class_tds[5].getText() == '◯':
        # print("부메랑")
        values += ("1",)
    else:
        values += ("0",)        
    # print(values)
    # End of scraping a CV
    # Write to server
    values_list.append(values)
print(len(values_list))
print(values_list[0])
print(values_list[-1])

In [18]:
# 한글 번역 이름 scraping 수동으로 번역하기
soup = BeautifulSoup(driver.page_source, 'html.parser')

tbody = soup.select('body > div.l-content.js-end-trigger > div.l-3col > div.l-3colMain > div.l-3colMain__center.l-3colMain__center--shadow > div.archive-style-wrapper > div.a-tabContainer > div.a-tabPanels > div.a-tabPanel.is-active > table > tbody')
vctrs = tbody[0].select('tbody > tr')
vc_trsl_name_list = []
for cnt in range(0, int(len(vctrs)/7)):
    start = 7*cnt
    end = 7*(cnt+1)
    vctr = vctrs[start:end]
    name_tr = vctr[1]
    vc_trsl_name_list.append(name_tr.find('td').getText())
for i in range(len(vc_trsl_name_list)):
    values_list[i] = (vc_trsl_name_list[i], *values_list[i])
print(len(vc_trsl_name_list))
print(values_list[0])
print(values_list[-1])

52
('해변의 승자', '浜辺の勝者', 'https://game8.jp/geneisensou/543354', 'https://img.game8.jp/8459955/230af791f47da2d6888d6cfed2e122b7.png/show', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '1', '0')


In [19]:
# update VC for JOB to server.
sql = f"""INSERT INTO `ffbe`.`{table_name}`
    (`vc_trsl_name`,`vc_jp_name`,`vc_g8_link`, `vc_img_src`,`검1`,`검2`,`검3`,`지팡이1`,`지팡이2`,`대검`,`창`,`도끼`,`활`,`총`,`주먹`,`단검`,`닌자`,`도검`,`메이스`,`글로브`,`책`,`부메랑`) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
write_to_server(sql,values_list)
# 서버에 올라간 내용을 엑셀파일로도 저장.
table_name = "vc_for_job_for_update_list"
excel_file_name = "vc_for_job_from_web.xlsx"

import pandas as pd
sql_query = f"SELECT * from {table_name}"
data = fetch_data_from_db(sql_query)
df = pd.DataFrame(data)
df.to_excel(excel_file_name, index=False)

[]

In [3]:
# Char_list Scraping starts from here.
driver = webdriver.Chrome()
driver.get('https://game8.jp/geneisensou/299431')

In [6]:
# 일본 이름 가지고 오기
# bs4로 html 읽어옴. 
char_soup = BeautifulSoup(driver.page_source, 'html.parser')
tr_soups = char_soup.select('#search-result > table > tbody > tr')
tr_soups.__delitem__(0)
tr_soups[0].contents[4].contents[0].text # '英雄 ' 나타나야 함.

# Scraping char data
char_jp_names = []
cnt = 0
for tr in tr_soups:
    td_elements = tr.select('tr > td')
    char_jp_names.append(td_elements[0].text)
    cnt+=1
print("총 캐릭수: ", len(char_jp_names))
print("First: ", char_jp_names[0])
print("Last: ", char_jp_names[-1])

총 캐릭수:  215
First:  セフィロス
Last:  ミーア


In [9]:
# 한글로 번역해 둘 것
char_soup = BeautifulSoup(driver.page_source, 'html.parser')
tr_soups = char_soup.select('#search-result > table > tbody > tr')
tr_soups.__delitem__(0)

# Scraping char data
values_list = []
char_trsl_name = None 
char_jp_name = None # To be used later
char_g8_link = None
char_img_src = None
char_eval = None
char_rarity = None
char_element = None
char_element_img_src = None
char_main_job = None
char_sub_job1 = None
char_sub_job2 = None

for i, tr in enumerate(tr_soups):
    td_elements = tr.select('tr > td')
    char_trsl_name = (td_elements[0].text)
    char_g8_link = td_elements[0].find('a')['href']
    char_img_src = (td_elements[0].find('img').get('src'))
    char_eval = (td_elements[1].text)
    char_rarity = (td_elements[2].text)
    char_element = (td_elements[3].text)
    char_element_img_src = (td_elements[3].find('img').get('src'))
    char_main_job = (td_elements[4].contents[0].text)
    try:
        char_sub_job1 = (td_elements[4].contents[2].text)
        char_sub_job2 = (td_elements[4].contents[4].text)
    except:
        print(f"Exception with {td_elements[4].contents[0].text}")
        char_sub_job1 = (td_elements[4].contents[0].text)
        char_sub_job2 = (td_elements[4].contents[0].text)
    values_list.append([char_jp_names[i], char_trsl_name,char_g8_link,char_img_src,char_eval,char_rarity,char_element,char_element_img_src,char_main_job,char_sub_job1,char_sub_job2])
print("총 캐릭수: ", len(values_list))
print(f"First: {values_list[0]}")
print(f"Last: {values_list[-1]}")

Exception with 슬라임
총 캐릭수:  215
First: ['セフィロス', '세피로스', 'https://game8.jp/geneisensou/492425', 'https://img.game8.jp/7440558/1cb962045da567187db248f0a6996de6.png/show', '9.9', 'UR', '어둠', 'https://img.game8.jp/6380415/2e26b9aaf2c7a2946c0262e2bc242e62.png/show', '영웅', '솔저', '격투사']
Last: ['ミーア', '미어', 'https://game8.jp/geneisensou/301232', 'https://img.game8.jp/3833826/6f1f18c94d52ac107e2aecbdd0f0b2a7.png/show', '4.0', '아르 자형', '바람', 'https://img.game8.jp/6380413/1d4a460abf33a86dc2d1c679104deb90.png/show', '시프', '클레릭', '마법검사']


In [24]:
# Write to DB.
sql = """
truncate char_list_for_update;
"""
fetch_data_from_db(sql)
sql2 = """
INSERT INTO char_list_for_update (char_jp_name,char_trsl_name,char_g8_link,char_img_src,char_eval,char_rarity,char_element,char_element_img_src,char_main_job,char_sub_job1,char_sub_job2)
values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
write_to_server(sql2, values_list)
sql = """
update char_list_for_update
set char_element = 
	CASE char_element
		WHEN '어둠' THEN '암'
        WHEN '흙' THEN '토'
        WHEN '빛' THEN '명'
        WHEN '우뢰' THEN '뇌'
		WHEN '뢰' THEN '뇌'
        WHEN '얼음' THEN '빙'
        WHEN '바람' THEN '풍'
        WHEN '물' THEN '수'
        WHEN '불' THEN '화'
        ELSE char_element
	END
 """
write_to_server(sql)
sql = """
update char_list_for_update
set char_main_job_class = 
	(select class_name from job_list where job_trsl_name = char_main_job),
	char_main_job_class_alias = 
	(select class_name_alias from job_list where job_trsl_name = char_main_job);
 """
write_to_server(sql)
#끝!!

[]

In [16]:
#Update class list manually
file_path = "./ffbe_vc_class_list_by_manual_update.xlsx"
df = pd.read_excel(file_path)
ddf = df.drop_duplicates(subset=["class_name_alias", "char_main_job"])
ddf = ddf.iloc[:,1:3]
ddf
sql = 'truncate job_list_for_update'
write_to_server(sql)
sql = """INSERT INTO job_list_for_update (job_trsl_name, class_name_alias)
values (%s, %s)"""
write_to_server(sql, ddf.values.tolist())
sql = """UPDATE job_list_for_update as u SET
class_name = (select distinct class_name from job_list as o where o.class_name_alias = u.class_name_alias)"""
write_to_server(sql)

[]

In [ ]:
table_name = "job_list"
sql=f"""
select * from {table_name}
"""
data, col_names = fetch_data_from_db_with_col_names(sql)
df = pd.DataFrame(data, columns=col_names)
df

In [26]:
ddf = df.drop_duplicates(subset=["job_trsl_name", "class_name_alias"])
ddf = ddf.iloc[:,2:]
ddf.to_excel(f"{table_name}.xlsx", index=False)

In [9]:
#파일을 df로 불러와서 server에 저장. 원래 있던 내용은 TRUNCATE 시키기.
table_name = "job_list"
file_path = table_name + '.xlsx'
df = pd.read_excel(file_path)
columns = df.columns.tolist()

sql = f"""TRUNCATE {table_name}"""
write_to_server(sql)

sql = f"""INSERT INTO {table_name} ({','.join(columns)})
VALUES ({','.join(['%s']*len(columns))})
"""
values_list = []
for _, d in df.iterrows():
    values_list.append(d.values.tolist())
write_to_server(sql, values_list)
#끝!

In [42]:
page_source = driver.page_source
job_soup = BeautifulSoup(page_source, 'html.parser')

In [51]:
job_soup = BeautifulSoup(page_source, 'html.parser')
tbody = job_soup.select('body > div.l-content.js-end-trigger > div.l-3col > div.l-3colMain > div.l-3colMain__center.l-3colMain__center--shadow > div.archive-style-wrapper > div.a-tabContainer > div.a-tabPanels > div.a-tabPanel.is-active > div > div > table > tbody')
trs = tbody[0].select('tbody > tr')

In [ ]:
# job_list와 class_list를 구축

# Connect to the MySQL database
db_connection = mysql.connector.connect(
    host='146.56.43.43',
    user='ffbemaster',
    password='aksaksgo1!',
    database='ffbe')

cursor = db_connection.cursor()

tbody = job_soup.select('body > div.l-content.js-end-trigger > div.l-3col > div.l-3colMain > div.l-3colMain__center.l-3colMain__center--shadow > div.archive-style-wrapper > div.a-tabContainer > div.a-tabPanels > div.a-tabPanel.is-active > div > div > table > tbody')
trs = tbody[0].select('tbody > tr')

cnt = 0
for tr in trs:
    cnt += 1
    if tr.find('th'):
        try:
            class_name = tr.text.strip().split('=')[1].strip()
            class_name2 = tr.text.strip().split('=')[0].strip()
        except:
            class_name = tr.text.strip().split('=')[0].strip()
            class_name2 = tr.text.strip().split('=')[0].strip()
        print(f"Class category: {class_name} or {class_name2}")
        sql = "INSERT IGNORE INTO class_list (class_name, class_alias) values (%s, %s)"
        values = (class_name, class_name2)
        cursor.execute(sql, values)
    else:
        job_names = tr.text.strip().splitlines()
        for j in job_names:
            print(j)
            sql = "INSERT IGNORE INTO job_list (job_name, class_name, class_alias) values (%s,%s,%s)"
            values = (j.strip(), class_name, class_name2)
            cursor.execute(sql, values)
db_connection.commit()

In [ ]:
# Server의 char_list에서 main_job에 따른 무구직종 분류 추가

# Connect to the MySQL database
db_connection = mysql.connector.connect(
    host='146.56.43.43',
    user='ffbemaster',
    password='aksaksgo1!',
    database='ffbe')

cursor = db_connection.cursor()
sql = "SELECT char_name, char_main_job FROM char_list"
cursor.execute(sql)

results = cursor.fetchall()
for r in results:
    char_name = r[0]
    sql2 = "SELECT job_name, class_name, class_alias FROM job_list where job_name = %s"
    values = (r[1],)
    cursor.execute(sql2, values)
    for r2 in cursor.fetchall():
        print(r2)
        sql3 = "UPDATE char_list SET char_main_job_class1 = %s, char_main_job_class2 = %s WHERE char_name = %s"
        values = (r2[1], r2[2], char_name)
        cursor.execute(sql3, values)
db_connection.commit()

In [ ]:
# 로컬 csv 데이터를 server에 업로드(update)
import pandas as pd
import mysql.connector
import numpy as np

db_connection = mysql.connector.connect(
    host='146.56.43.43',
    user='ffbemaster',
    password='aksaksgo1!',
    database='ffbe')

# Read data from the local file
data_file = 'ffbe_char_list_with_null.csv'
df = pd.read_csv(data_file, encoding='cp949')
df = df.replace(np.nan, None)
# Connect to the MySQL server
try:
    cursor = db_connection.cursor()

    # Generate the update SQL query dynamically
    table_name = "char_list"
    update_columns = ', '.join([f"{col} = %s" for col in df.columns])
    sql = f"UPDATE {table_name} SET {update_columns} WHERE seq = %s"

    # Loop through each row in the DataFrame and update the corresponding records in the MySQL table
    for index, row in df.iterrows():
        values = tuple(row.values)  # Convert the row values to a tuple
        # Append the 'id' value at the end of the tuple
        values += (row['seq'],)
        print(values)
        cursor.execute(sql, values)

    # Commit the changes and close the connection
    db_connection.commit()
    cursor.close()
    db_connection.close()
    print("Data update successful!")

except mysql.connector.Error as e:
    print(f"Error: {e}")

In [159]:
# Connect to the MySQL database
db_connection = mysql.connector.connect(
    host='146.56.43.43',
    user='ffbemaster',
    password='aksaksgo1!',
    database='ffbe')

cursor = db_connection.cursor()
sql = "DESC vc_class_tb"
# sql = "SELECT * FROM vc_class_tb"
cursor.execute(sql)


In [ ]:
response = cursor.fetchall()
response

In [ ]:
INSERT INTO `ffbe`.`vc_class_tb`
(`vc_name`,`vc_img_src`,`검1`,`검2`,`검3`,`지팡이1`,`지팡이2`,`대검`,`창`,`도끼`,`활`,`총`,`주먹`,`단검`,`닌자`,`도검`,`메이스`,`글로브`,`책`,`부메랑`) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
tbody = soup.select('body > div.l-content.js-end-trigger > div.l-3col > div.l-3colMain > div.l-3colMain__center.l-3colMain__center--shadow > div.archive-style-wrapper > div.a-tabContainer > div.a-tabPanels > div.a-tabPanel.is-active > table > tbody')
vctrs = tbody[0].select('tbody > tr')

# Connect to the MySQL database
db_connection = mysql.connector.connect(
    host='146.56.43.43',
    user='ffbemaster',
    password='aksaksgo1!',
    database='ffbe')
    
for cnt in range(1, 49):
    start = 7*cnt
    end = 7*(cnt+1)
    vctr = vctrs[start:end]
    name_tr = vctr[1]
    vc_name = name_tr.find('td').getText()
    vc_img_src = name_tr.find('img')['src']
    values = (vc_name, vc_img_src)
    print(vc_name)
    print(vc_img_src)
    class_tds = vctr[2].findAll('td')
    if class_tds[0].getText() == '◯':
        print("검1")
        values += ("1",)
    else:
        values += ("0",)
    if class_tds[1].getText() == '◯':
        print("검2")
        values += ("1",)
    else:
        values += ("0",)
    if class_tds[2].getText() == '◯':
        print("검3")                
        values += ("1",)
    else:
        values += ("0",)    
    if class_tds[3].getText() == '◯':
        print("지팡이1")                
        values += ("1",)
    else:
        values += ("0",)
    if class_tds[4].getText() == '◯':
        print("지팡이2")                
        values += ("1",)
    else:
        values += ("0",)
    if class_tds[5].getText() == '◯':
        print("대검")                
        values += ("1",)
    else:
        values += ("0",)
    
    class_tds = vctr[4].findAll('td')
    if class_tds[0].getText() == '◯':
        print("창")
        values += ("1",)
    else:
        values += ("0",)
    if class_tds[1].getText() == '◯':
        print("도끼")
        values += ("1",)
    else:
        values += ("0",)
    if class_tds[2].getText() == '◯':
        print("활")                
        values += ("1",)
    else:
        values += ("0",)
    if class_tds[3].getText() == '◯':
        print("총")                
        values += ("1",)
    else:
        values += ("0",)
    if class_tds[4].getText() == '◯':
        print("주먹")                
        values += ("1",)
    else:
        values += ("0",)
    if class_tds[5].getText() == '◯':
        print("단검")                
        values += ("1",)
    else:
        values += ("0",)    

    class_tds = vctr[6].findAll('td')
    if class_tds[0].getText() == '◯':
        print("닌자")
        values += ("1",)
    else:
        values += ("0",)
    if class_tds[1].getText() == '◯':
        print("도검")
        values += ("1",)
    else:
        values += ("0",)
    if class_tds[2].getText() == '◯':
        print("메이스")                
        values += ("1",)
    else:
        values += ("0",)    
    if class_tds[3].getText() == '◯':
        print("글로브")                
        values += ("1",)
    else:
        values += ("0",)
    if class_tds[4].getText() == '◯':
        print("책")                
        values += ("1",)
    else:
        values += ("0",)
    if class_tds[5].getText() == '◯':
        print("부메랑")
        values += ("1",)
    else:
        values += ("0",)        
    print(values)

    cursor = db_connection.cursor()
    sql = """INSERT INTO `ffbe`.`vc_class_tb`
    (`vc_name`,`vc_img_src`,`검1`,`검2`,`검3`,`지팡이1`,`지팡이2`,`대검`,`창`,`도끼`,`활`,`총`,`주먹`,`단검`,`닌자`,`도검`,`메이스`,`글로브`,`책`,`부메랑`) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    cursor.execute(sql, values)
db_connection.commit()    
cursor.close()
db_connection.close()

In [154]:
values = ("a", "b")
values += ("c",)
values

('a', 'b', 'c')